In [2]:
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import scipy.signal as signal
from myFilter import Filter as Filter
import os
plt.style.use('seaborn-whitegrid')

def calFFT(signal, window = 2048 , shift = False , inDB = False, half = True, normf=True, fs=None):
    
    from scipy.fftpack import fft, fftshift
    mag = np.abs(fft(signal, window) / (len(signal)/2.0))
    freq = np.linspace(0, 1, len(mag))

    if shift:
        mag = np.abs(fftshift(mag / abs(mag).max() ) )
        freq = np.linspace(-0.5, 0.5, len(mag))
        
    
    if inDB:
        mag = 20 * np.log10( mag )

    if normf == False:
        if fs == None:
            raise ValueError("Give me 'fs'")
        freq = np.linspace(0, fs, len(mag) )

    if half:
        mag = mag[:len(mag)//2]
        freq = freq[:len(freq)//2]

    return mag, freq

def matInfo(mat):
    print(mat.keys())
    for key in mat.keys():
        print(f"mat['{key}'] - {mat[key]}")

samplingRate = 1000 #hz

In [3]:
# Load mat
bands_name = {
    0 : '0_delta_theta',
    1 : '1_alpha',
    2 : '2_low_beta',
    3 : '3_high_beta',
    4 : '4_low_gamma',
    5 : '5_middle_gamma',
    6 : '6_high_gamma'
}
folder_name = f"EEG"

# Since I have to always load the mat file and squeez out the extra layer, let's just make a function for it
def get_signal(mat, key):
    return mat[key][0]

# This is how  to read the mat file
eeg_mat = scipy.io.loadmat('EEG/EEG_rest.mat')
# The original channel 0
eeg_0 = get_signal(eeg_mat, '0')
# Channel 0 with highpass
eeg_0_highpass = get_signal(eeg_mat, '0_highpass')
# Delta + Theta component on Channel 0
eeg_0_0_delta_theta = get_signal(eeg_mat, '0_0_delta_theta')
print(eeg_0_0_delta_theta.shape)

(300000,)


In [4]:
# Prepare data

# Let's get delta theta of all channel 
delta_theta = eeg_mat[f"0_0_delta_theta"]
for i in range(15):
    temp = eeg_mat[f"{i+1}_0_delta_theta"]
    delta_theta = np.concatenate([delta_theta,temp], axis=0)

print(delta_theta.shape)
# (16, 300000) is n_features, n_samples

bss = dict()

(16, 300000)


In [5]:
#  Load ECoG for Correlation Coefficients calculation
ECoG = scipy.io.loadmat('dataset/20120904S11_EEGECoG_Chibi_Oosugi-Naoya+Nagasaka-Yasuo+Hasegawa+Naomi_ECoG128-EEG16_mat/ECoG_rest.mat')
ECoG = ECoG['ECoG']
# (128, 300000)

In [6]:
def cal_correlation_coefficient(corr,mode='max',abs=False):
    import numpy as np
    _list_mode = ['max','sum']
    if(type(corr) != type(np.array([]))): corr = np.array(corr)
    if(mode not in _list_mode): raise ValueError(f"mode can only be {_list_mode}")
    if(abs): corr = np.abs(corr)
    # print(corr)
    if(mode == 'max'): return corr.max(axis=1)
    if(mode == 'sum'): return corr.sum(axis=1)

corr_mode = 'sum'
corr_abs = True
# cal_correlation_coefficient(R[:X_chs,X_chs:X_chs+Y_chs],mode='sum',abs=False)

# PCA

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
# X: array-like, shape (n_samples, n_features)
# Training data, where n_samples is the number of samples and n_features is the number of features.
# y: None
# Ignored variable.
pca_delta_theta = pca.fit_transform(delta_theta.T).T
# Check if the algorithm works
assert (pca_delta_theta == delta_theta).all() == False, f"You data is the same."
pca_delta_theta.shape
# (16, 300000)
bss['pca'] = pca_delta_theta

In [8]:
R = np.corrcoef(pca_delta_theta, ECoG)
pca_corr = cal_correlation_coefficient(R[:16,16:16+128],mode=corr_mode,abs=corr_abs)
bss['pca_corr'] = pca_corr

# ILRMA

In [9]:
import pyroomacoustics as pra

## Prepare one-shot STFT
L = 2048
hop = L // 2
win_a = pra.hann(L)
win_s = pra.transform.stft.compute_synthesis_window(win_a, hop)

## STFT ANALYSIS
# (n_samples, n_channels)
X = pra.transform.stft.analysis(delta_theta.T, L, hop, win=win_a)

# Run ILRMA
Y = pra.bss.ilrma(X, n_iter=100, n_components=16, proj_back=True) #callback=convergence_callback)
ilrma_delta_theta = pra.transform.stft.synthesis(Y, L, hop, win=win_s).T
print(ilrma_delta_theta.shape)
# (16, 300032)
ilrma_delta_theta = ilrma_delta_theta[:,:300000]
# (16, 300000)

(16, 300032)


In [10]:
R = np.corrcoef(ilrma_delta_theta[:,:300000], ECoG)
ilrma_corr = cal_correlation_coefficient(R[:16,16:16+128],mode=corr_mode,abs=corr_abs)
bss['ilrma_corr'] = ilrma_corr

# CCA

In [11]:
# https://www.csee.umbc.edu/~liyiou1/li_ICASSP08.pdf
# https://ieeexplore.ieee.org/document/8078739
# https://towardsdatascience.com/understanding-how-schools-work-with-canonical-correlation-analysis-4c9a88c6b913
from sklearn.cross_decomposition import CCA

cca = CCA(n_components=16)
# Input:::
# X: array-like of shape (n_samples, n_features)
# Training vectors, where n_samples is the number of samples and n_features is the number of predictors.
# y: array-like of shape (n_samples, n_targets)
# Target vectors, where n_samples is the number of samples and n_targets is the number of response variables.

# Return:::
# x_scores if Y is not given, (x_scores, y_scores) otherwise.

cca_delta_theta, cca_ECoG = cca.fit_transform(delta_theta.T, ECoG.T)

In [38]:
# print(cca.x_scores_)
# print(cca_delta_theta)
# cca_delta_theta.shape
# cca_ECoG.shape
# cca.y_scores_.shape

# Although cca_delta_theta != x_scores and cca_EcoG != y_scores
# and c1 != c2
# but the cal_correlation is the same in mode='sum' abs=True

# In this case, I choose to do this using the cca_delta_theta
c1 = np.corrcoef(cca.x_scores_.T, cca.y_scores_.T)
c2 = np.corrcoef(cca_delta_theta.T,cca_ECoG.T)

print(cal_correlation_coefficient(c1[:16,16:16+16],mode=corr_mode,abs=corr_abs))
print(cal_correlation_coefficient(c2[:16,16:16+16],mode=corr_mode,abs=corr_abs))

cca_delta_theta = cca_delta_theta.T
cca_delta_theta.shape
# (16, 300000)

[0.24969898 0.237256   0.16976499 0.15272714 0.14695348 0.13794119
 0.12767575 0.12620091 0.11991308 0.11236153 0.10022023 0.09527144
 0.09283677 0.08669956 0.08164268 0.07027476]
[0.24969898 0.237256   0.16976499 0.15272714 0.14695348 0.13794119
 0.12767575 0.12620091 0.11991308 0.11236153 0.10022023 0.09527144
 0.09283677 0.08669956 0.08164268 0.07027476]


(16, 300000)

In [39]:
R = np.corrcoef(cca_delta_theta, ECoG)
cca_corr = cal_correlation_coefficient(R[:16,16:16+128],mode=corr_mode,abs=corr_abs)
bss['cca_corr'] = cca_corr

In [40]:
print(pca_corr)
print(ilrma_corr)
print(cca_corr)

[1.11620275 1.92321668 1.57905784 1.06816192 1.42210329 1.58783369
 1.31137826 1.27184437 1.52574255 0.91003249 1.21193236 1.09855706
 1.90151398 1.22922325 1.32471476 1.39643461]
[0.87521983 0.86892372 0.78809889 0.89857807 0.80455041 0.92990338
 0.84817853 1.14544763 0.9042692  0.98178423 0.84400103 0.8025505
 1.16515205 1.23065506 0.74241364 1.03063727]
[2.24640275 2.79993629 1.70613402 1.11679859 1.05274363 1.36338489
 1.19940576 1.18514694 1.27250003 1.33580075 0.88262981 0.99754068
 1.10933771 0.99603263 1.23702154 0.57014543]


In [41]:
# To prove that sklearn CCA is correct, here is another CCA algor which yeild the same CC score list

# https://devdocs.io/statsmodels/generated/statsmodels.multivariate.cancorr.cancorr
from statsmodels.multivariate.cancorr import CanCorr

temp_out = CanCorr(ECoG.T, delta_theta.T)
result = temp_out.corr_test()
print(result)

                              Cancorr results
   Canonical Correlation Wilks' lambda   Num DF     Den DF    F Value Pr > F
----------------------------------------------------------------------------
0                 0.2497        0.7244 2048.0000 4759678.7611 47.6708 0.0000
1                 0.2370        0.7726 1905.0000 4465902.6950 40.9544 0.0000
2                 0.1696        0.8186 1764.0000 4171495.9521 34.2665 0.0000
3                 0.1527        0.8428 1625.0000 3876479.1415 31.7533 0.0000
4                 0.1468        0.8629 1488.0000 3580875.1474 29.8824 0.0000
5                 0.1377        0.8819 1353.0000 3284709.2784 27.9955 0.0000
6                 0.1276        0.8990 1220.0000 2988009.4253 26.3073 0.0000
7                 0.1260        0.9139 1089.0000 2690806.2242 24.9223 0.0000
8                 0.1198        0.9286  960.0000 2393133.2268 23.2397 0.0000
9                 0.1122        0.9421  833.0000 2095027.0745 21.5471 0.0000
10                0.1002      

# Save to MAT

In [42]:
if(os.path.exists('BSS') == False):
    os.mkdir('BSS')
    
scipy.io.savemat('BSS/step2.mat',bss)